In [2]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
documents[2]

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

In [4]:
import minsearch

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [9]:
from openai import OpenAI
from dotenv import load_dotenv
import os
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI()

In [10]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [11]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [12]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [13]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [14]:
rag('how do I run kafka?')

"To run Kafka in the terminal, for Java Kafka, navigate to your project directory and execute the following command:\n\n```bash\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nFor running a Python Kafka producer, it's recommended to create and activate a virtual environment, install the required packages, and ensure your Docker images are up and running. Here's the process to set up the virtual environment:\n\n1. Create a virtual environment:\n   ```bash\n   python -m venv env\n   ```\n\n2. Activate the virtual environment (on MacOS and Linux):\n   ```bash\n   source env/bin/activate\n   ```\n\n   On Windows, use:\n   ```bash\n   env\\Scripts\\activate\n   ```\n\n3. Install the required packages:\n   ```bash\n   pip install -r ../requirements.txt\n   ```\n\nRemember to deactivate the virtual environment when done using:\n```bash\ndeactivate\n```"

In [ ]:
rag('the course has already started, can I still enroll?')

"Yes, you can still enroll in the course after it has started. Even if you haven't registered, you're eligible to submit the homeworks. However, keep in mind that there are deadlines for submitting the final projects, so it's advisable not to delay everything until the last minute."

: 